# State reconstruction tutorial

* Create package objects required for state estimation
* Reconstruct site-resolved fluorescence
* **Make sure you have run the tutorial `image_generation.ipynb`**

In [ ]:
import state_reconstruction as srec

import json
import os

from libics.env import DIR_DESKTOP
from libics.core import io
from libics.tools import plot

## Package configuration

**The `state_reconstruction` package configuration**

* can be read as follows
* and can be altered by overwriting the appropriate parameters in the configuration file located at `~/.libics/state_estimation/config.json`, where `~` indicates the user folder (e.g. `C:/Users/<my_user_name>`

In [ ]:
srec.get_config()

**Files generated using this tutorial**

* are saved to the desktop with the following file name
* and may be used by other tutorials

In [ ]:
DEMO_FILENAME = "srec_demo"

## Load prerequisites

**Load affine transformation object**

In [ ]:
filepath_trafo_site_to_image = os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_trafo.json")
trafo_site_to_image = io.load(filepath_trafo_site_to_image)
trafo_site_to_image

In [ ]:
phase_ref_image = srec.get_config("trafo_gen.phase_ref_image")
phase_ref_site = srec.get_config("trafo_gen.phase_ref_site")
phase_ref_image, phase_ref_site

**Load PSF object**

In [ ]:
filepath_ipsf_gen = os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_psf.json")
ipsf_gen = srec.IntegratedPsfGenerator.load(filepath_ipsf_gen)
ipsf_gen

**Load image to be reconstructed**

In [ ]:
demo_image = io.load(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_image-dense.png")
)
demo_occ = io.load(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_occ-dense.json")
)

## Image preprocessor

**Create demo image with outliers**

* Occasionally, the camera sensor picks up a high-energy event, resulting in a local outlier
* The `ImagePreprocessor` object removes the affected areas to enable reconstruction of the rest of the image

In [ ]:
demo_image_outlier = demo_image.copy()
demo_image_outlier[150:153, 200:202] += 1e4

plt_roi = (slice(130, -130), slice(130, -130))

fig, axs = plot.subplots(figsize=(11, 3), ncols=3)
plot.pcolorim(
    demo_image_outlier[plt_roi], ax=axs[0], title="w/ outlier (full scale)",
    cmap="hot", vmin=0, vmax=None, colorbar=True,
)
plot.pcolorim(
    demo_image_outlier[plt_roi], ax=axs[1], title="w/ outlier (reduced scale)",
    cmap="hot", vmin=0, vmax=1100, colorbar=True,
)
plot.pcolorim(
    demo_image[plt_roi], ax=axs[2], title="w/o outlier (full scale)",
    cmap="hot", vmin=0, vmax=1100, colorbar=True,
)
plot.style_figure(tight_layout=True)

**Create image preprocessor object**

In [ ]:
img_preproc = srec.ImagePreprocessor()
img_preproc

**Demo for outlier removal**

In [ ]:
demo_image_outlier_removed, outlier_ratio = (
    img_preproc.process_image(demo_image_outlier)
)

print(outlier_ratio)
plt_roi = (slice(130, -130), slice(130, -130))

fig, axs = plot.subplots(figsize=(11, 3), ncols=3)
plot.pcolorim(
    demo_image_outlier[plt_roi], ax=axs[0], title="w/ outlier (full scale)",
    cmap="hot", vmin=0, vmax=None, colorbar=True,
)
plot.pcolorim(
    demo_image_outlier_removed[plt_roi], ax=axs[1], title="rmvd. outlier (full scale)",
    cmap="hot", vmin=0, vmax=None, colorbar=True,
)
plot.pcolorim(
    demo_image[plt_roi], ax=axs[2], title="w/o outlier (full scale)",
    cmap="hot", vmin=0, vmax=1100, colorbar=True,
)
plot.style_figure(tight_layout=True)

## Projector generator

**Instantiate the projector generator object**

* This takes into account overlapping PSFs
* and projects the emitted fluorescence of each lattice site into site space

In [ ]:
proj_gen = srec.ProjectorGenerator(
    trafo_site_to_image=trafo_site_to_image,
    integrated_psf_generator=ipsf_gen,
    proj_shape=(61, 61)
)
proj_gen

**Set up cache**

* Calculating the projection matrices is a time-consuming process
* The `state_reconstruction` package therefore pre-calculates the required projectors
* The cache directory can be set in the package configuration

In [ ]:
# Check projector cache directory
srec.get_config("projector_cache_dir")

In [ ]:
# Pre-calculate projectors (this may take up to a few minutes)
proj_gen.setup_cache(print_progress=True)

# If you execute this cell again, this should run instantaneously due to caching

**Demo projector**

* Calculating the scalar product between projector and image yields the emission of a given site
* To account for overlapping PSFs, the projector also contains negative values

In [ ]:
demo_proj = proj_gen.generate_projector()

plot.pcolorim(
    demo_proj,
    cmap="RdBu_r", vcen=0, vdif=True, colorbar=True
)

## Emission histogram analysis

**Create emission histogram analysis object**

* After projection, we obtain a fluorescence emission value for each lattice site
* We then have to distinguish filled from non-filled sites
* This is achieved by analyzing a histogram comprising fitting the background and emission peaks

In [ ]:
eha = srec.EmissionHistogramAnalysis()
eha

## Perform reconstruction

**Create state estimator object**

* This object automatically handles all previous steps to reconstruct the site-resolved emission state

In [ ]:
sites_shape = (170, 170)

sest = srec.StateEstimator(
    id=DEMO_FILENAME,
    image_preprocessor=img_preproc,
    phase_ref_image=phase_ref_image,
    phase_ref_site=phase_ref_site,
    trafo_site_to_image=trafo_site_to_image,
    projector_generator=proj_gen,
    sites_shape=sites_shape,
    emission_histogram_analysis=eha,
)
sest

**Reconstruct sample images**

* (Please ignore warnings from the `libics.tools.math` modules)

In [ ]:
# Perform reconstruction
res = sest.reconstruct(demo_image)

# Plot reconstruction results
_ = srec.plot_reconstruction_results(
    sest, res, demo_image
)

In [ ]:
# Check reconstruction fidelity
fig, axs = plot.subplots(figsize=(14, 3), ncols=4)
plt_roi = (slice(65, -65), slice(65, -65))

plot.pcolorim(
    res.emissions[plt_roi], ax=axs[0],
    colorbar=True, vmin=0, title="Projected emissions"
)
plot.pcolorim(
    res.state[plt_roi], ax=axs[1],
    colorbar=True, vmin=0, vmax=2, title="Reconstructed occupation"
)
plot.pcolorim(
    demo_occ[plt_roi], ax=axs[2],
    colorbar=True, vmin=0, vmax=2, title="Actual occupation"
)
plot.pcolorim(
    (res.state - demo_occ)[plt_roi], ax=axs[3],
    cmap="RdBu_r", colorbar=True, vcen=0, vdif=1, title="Occupation difference"
)
plot.style_figure(tight_layout=True)

**Save state estimator object**

* So far, the `StateEstimator` object cannot be automatically saved to a file yet
* However, it is possible to load the object from a configuration file
* Recursive object reconstruction is also supported:
  * This means that a dictionary can be passed for each required attribute
  * The dictionary is then used to construct the respective object
  * The passed values overwrite default values
  * For more complex objects, we can often alternatively pass a file path, from which the object is read
* An example configuration file for the object above would look as follows

In [ ]:
_sest_config_dict = {
    # ID of the state estimator object
    "id": sest.id,
    # Image preprocessor (default)
    "image_preprocessor": {
    },
    # Lattice phase reference
    "phase_ref_site": phase_ref_site,
    "phase_ref_image": phase_ref_image,
    # Trafo object
    "trafo_site_to_image": filepath_trafo_site_to_image,
    # Projector generator object
    "projector_generator": {
        "integrated_psf_generator": filepath_ipsf_gen,
        "proj_shape": sest.proj_shape
    },
    # Reconstructed sites shape
    "sites_shape": sites_shape,
    # Emission histogram analysis object (default)
    "emission_histogram_analysis": {
    }
}
_sest_config_dict

In [ ]:
# Write state estimator configuration file
with open(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_state-estimator.json"), "w"
) as _f:
    json.dump(_sest_config_dict, _f, indent=4)

In [ ]:
# Construct state estimator object from configuration file
sest = srec.StateEstimator.from_config(
    config=os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_state-estimator.json")
)
sest

**Default configuration directory**

* There exists a much more convenient directory to store the state estimator configuration files
* Configurations are then easily discoverable and can be loaded without specifying a full file path
* It is customary to set the configuration file name to the state estimator ID

In [ ]:
default_config_folder = srec.get_config("state_estimator_config_dir")
if not os.path.exists(default_config_folder):
    os.makedirs(default_config_folder)
default_config_folder

In [ ]:
# Write state estimator configuration file
with open(
    os.path.join(default_config_folder, sest.id + ".json"), "w"
) as _f:
    json.dump(_sest_config_dict, _f, indent=4)

In [ ]:
# Easily search for state estimator configurations
srec.StateEstimator.discover_configs()

In [ ]:
# Load state estimator object
srec.StateEstimator.from_config(config=sest.id)